# WebScarping 

 # Import Packages

In [1]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
from collections import defaultdict
import re
import json

In [2]:
url = "https://fangj.github.io/friends/"
html = urlopen(url)

In [3]:
soup = BeautifulSoup(html, 'lxml')
type(soup)

bs4.BeautifulSoup

In [4]:
# Print out the text
text = soup.get_text()

In [5]:
season_list = text.split('\n')

In [6]:
episodes =[l for l in season_list if l[0:3].isdigit()];
pattern1 ='\d{3}\s'
pattern2 = '\d{3}-\d{3}'
episode_numbers = [l [0:3] if re.match(pattern1,l) else l[0:7] for l in episodes]
episode_names = [l[4:] if re.match(pattern1,l) else l[8:] for l in episodes]

Except for season 724, Most common url formats are addressed as 3 digit or 7 digit format. we will do try and except for that one episode so that we can continue the loop

In [7]:
all_transcripts =   {'Episodes':[], 'Episode_Names':[],'Characters': [],'Lines': []} 
for episode in episode_numbers:
    try:
        # to extract url that has 3 digit as the number of the episode (e.g. 101)
        if len(episode) == 3:
            url1 = 'https://fangj.github.io/friends/season/0{0}.html'.format(str(episode))
            html = urlopen(url1)
        else:
            # to extract url that has 7 digit as the number of the episode (e.g. 212-213)
            url2 = 'https://fangj.github.io/friends/season/0{0}-0{1}.html'.format(episode[0:3],episode[4:])
            html = urlopen(url2)  
    # only one  url  of all the episodes had a diferent format than the above two in the url, ignore it
    except Exception:
        continue
            
    #parse the html
    soup = BeautifulSoup(html, 'lxml')
    #identify html tags and attributes associated with actors and their dialogues in the script
    transcripts = soup.find_all('p')

     # Print text only, and further filter tags to fin Charcater: Lines
    pattern = '^\w+[:]'   
    transcripts_text_only = [p.text for p in transcripts if re.search(pattern,p.text) ]

    #split Characters and Lines
    Characters_this_episode = [l.split(":") [0] for l in transcripts_text_only]
    Lines_this_episode = [l.split(":") [1] for l in transcripts_text_only]
    
    #store episode number and episode name for each line
    Episode = [episode]*len(Characters_this_episode) 
    Episode_Name = [episode_names[episode_numbers.index(episode)]]*len(Characters_this_episode)
    
    all_transcripts['Episodes'].append(Episode)
    all_transcripts['Episode_Names'].append(Episode_Name)
    all_transcripts['Characters'].append(Characters_this_episode)
    all_transcripts['Lines'].append(Lines_this_episode)
   

In [8]:
type(all_transcripts['Episodes'])

list

In [9]:
#Each dictionary has list of lists as values. Flatten them so that each key has a single list
import itertools
all_transcripts['Episodes']= list (itertools.chain.from_iterable(  all_transcripts['Episodes']))
all_transcripts['Episode_Names']= list (itertools.chain.from_iterable(  all_transcripts['Episode_Names']))
all_transcripts['Characters']= list (itertools.chain.from_iterable(  all_transcripts['Characters']))
all_transcripts['Lines']= list (itertools.chain.from_iterable(  all_transcripts['Lines']))


In [10]:
print(len(all_transcripts['Episodes']))
print(len(all_transcripts['Episode_Names']))
print(len(all_transcripts['Characters']))
print(len(all_transcripts['Lines']))

48944
48944
48944
48944


# Export dictionary of transcripts as JSON

In [11]:
json = json.dumps( all_transcripts)
Transcripts = open("Friends_transcripts.json","w")
Transcripts.write(json)
Transcripts.close()